# Capybara
## Steven Ng
### Developer Skill Share
### July 7, 2016

https://github.com/skng5/developers-skill-share

## Introduction

Capybara helps you test web applications by simulating how a real user would interact with your app. It is agnostic about the driver running your tests and comes with Rack::Test and Selenium support built in. WebKit is supported through an external gem.

Ref: https://github.com/teamcapybara/capybara

### Key Benefits

- **No setup** necessary for Rails and Rack application. Works out of the box.
- **Intuitive API** which mimics the language an actual user would use.
- **Switch the backend** your tests run against from fast headless mode
  to an actual browser with no changes to your tests.
- **Powerful synchronization** features mean you never have to manually wait
  for asynchronous processes to complete.
  
Ref: https://github.com/teamcapybara/capybara/blob/master/README.md#key-benefits

## Behavior Driven Development in RSpec

- Provides Features and Scenario blocks like Gerkin (Cucumber).
- Allows Given/When/Then like features 

# Install

Gemfile:

```ruby
group :development, :test do
  gem 'solr_wrapper', '>= 0.3'
  gem 'rspec-rails'
  gem 'database_cleaner'
  gem 'factory_girl_rails'
  gem 'capybara'
  gem 'simplecov', require: false
  gem 'guard-rspec', require: false
end
```

Generate an rspec feature spec

```bash
rails generate rspec:feature indices
```

An Index page feature spec

```ruby
require 'rails_helper'
require 'traject'
require 'traject/command_line'
require 'yaml'

RSpec.feature "Indices", type: :feature do
  let (:fixtures) {
    YAML.load_file("#{fixture_path}/features.yml")
  }

  feature "Home Page" do
    context "publicly available pages" do
      scenario "User visits home page" do
        visit '/'
        expect(page).to have_text "Welcome!"
        within("#facets") do
            expect(page).to have_text "Date"
        end
      end

    end
  end

  feature "Catalog" do
    let (:title) { "Academic freedom in an age of conformity" }
    let (:results_url) { "http://www.example.com/?utf8=%E2%9C%93&search_field=all_fields&q=Academic+freedom+in+an+age+of+conformity" }
    scenario "Search" do
      visit '/'
      fill_in 'q', with: title
      click_button 'Search'
      expect(current_url).to eq results_url
      within(".document-position-0 h3") do
        expect(page).to have_text title 
      end
      within(".document-metadata") do
        expect(page).to have_text "Resource Type:"
        expect(page).to have_text "Book and Print"
        expect(page).to have_text "Status/Location:"
      end
    end
  end

  feature "Document" do
    let (:item) {
      fixtures.fetch("simple_search")
    }

    let (:item_url) {
      "#{Capybara.default_host}/catalog/#{item['doc_id']}"
    }

    scenario "Search" do
      visit '/'
      fill_in 'q', with: item['title']
      click_button 'Search'
      expect(current_url).to eq item['url']
      within(".document-position-0") do
        click_link item['title']
        expect(current_url).to eq item_url
        within("h3") do
          expect(page).to have_text item['title']
        end
        click_link item['title']
      end
    end

    scenario "User visits a document directly" do
      visit "catalog/#{item['doc_id']}"
      expect(current_url).to eq item_url
      expect(page).to have_text(item['title'])
    end
  end

  feature "MARC Fields" do
    let (:item) {
      fixtures.fetch("title_statement")
    }

    scenario "User visits a document with full title statement" do
      visit "catalog/#{item['doc_id']}"
      expect(page).to have_text(item['title'])
    end

  end
end
```

## Demo

In [ ]:
$ bundle exec rake rspec[spec/features]

```bash
Test solr server running: http://localhost:8985/solr/#/blacklight-core-test

^C to stop

2017-07-07T13:57:59+00:00  INFO traject (2.3.3) executing with: `-c app/models/traject_indexer.rb spec/fixtures/marc_fixture.xml`
2017-07-07T13:57:59+00:00  INFO Reading from spec/fixtures/marc_fixture.xml
2017-07-07T13:57:59+00:00  INFO    Traject::SolrJsonWriter writing to 'http://127.0.0.1:8985/solr/blacklight-core-test/update/json' in batches of 100 with 1 bg threads
2017-07-07T13:57:59+00:00  INFO    Indexer with 1 processing threads, reader: Traject::MarcReader and writer: Traject::SolrJsonWriter
2017-07-07T13:58:00+00:00  INFO Traject::SolrJsonWriter sending commit to solr at url http://127.0.0.1:8985/solr/blacklight-core-test/update/json...
2017-07-07T13:58:00+00:00  INFO finished Indexer#process: 202 records in 1.677 seconds; 120.5 records/second overall.
2017-07-07T13:58:01+00:00  INFO traject (2.3.3) executing with: `-c app/models/traject_indexer.rb -x commit`
2017-07-07T13:58:01+00:00  INFO Sending commit to: http://127.0.0.1:8985/solr/blacklight-core-test/update?commit=true
2017-07-07T13:58:01+00:00  INFO <?xml version="1.0" encoding="UTF-8"?>
<response>
<lst name="responseHeader"><int name="status">0</int><int name="QTime">1</int></lst>
</response>

.....

Finished in 6.44 seconds (files took 4.79 seconds to load)
5 examples, 0 failures

Coverage report generated for RSpec to /var/www/tulcob/coverage. 129 / 582 LOC (22.16%) covered.
```

## Capybara Cheat Sheet

https://gist.github.com/zhengjia/428105

### Navigating
```ruby
    visit('/projects')
    visit(post_comments_path(post))
```

### Clicking links and buttons
```ruby
    click_link('id-of-link')
    click_link('Link Text')
    click_button('Save')
    click('Link Text') # Click either a link or a button
    click('Button Value')
```


### Interacting with forms
```ruby
    fill_in('First Name', :with => 'John')
    fill_in('Password', :with => 'Seekrit')
    fill_in('Description', :with => 'Really Long Text…')
    choose('A Radio Button')
    check('A Checkbox')
    uncheck('A Checkbox')
    attach_file('Image', '/path/to/image.jpg')
    select('Option', :from => 'Select Box')
```

### Scoping
```ruby
    within("//li[@id='employee']") do
      fill_in 'Name', :with => 'Jimmy'
    end
    within(:css, "li#employee") do
      fill_in 'Name', :with => 'Jimmy'
    end
    within_fieldset('Employee') do
      fill_in 'Name', :with => 'Jimmy'
    end
    within_table('Employee') do
      fill_in 'Name', :with => 'Jimmy'
    end
```

### Querying
```ruby
    page.has_xpath?('//table/tr')
    page.has_css?('table tr.foo')
    page.has_content?('foo')
    page.should have_xpath('//table/tr')
    page.should have_css('table tr.foo')
    page.should have_content('foo')
    page.should have_no_content('foo')
    find_field('First Name').value
    find_link('Hello').visible?
    find_button('Send').click
    find('//table/tr').click
    locate("//*[@id='overlay'").find("//h1").click
    all('a').each { |a| a[:href] }
```

### Scripting
```ruby
    result = page.evaluate_script('4 + 4');
```

### Debugging
```ruby
    save_and_open_page
```

### Asynchronous JavaScript
```ruby
    click_link('foo')
    click_link('bar')
    page.should have_content('baz')
    page.should_not have_xpath('//a')
    page.should have_no_xpath('//a')
```

### XPath and CSS
```ruby
    within(:css, 'ul li') { ... }
    find(:css, 'ul li').text
    locate(:css, 'input#name').value
    Capybara.default_selector = :css
    within('ul li') { ... }
    find('ul li').text
    locate('input#name').value
```

## References

- [Capybara](http://www.rubydoc.info/gems/capybara)
- [RSpec Feature Specs](https://relishapp.com/rspec/rspec-rails/v/3-4/docs/feature-specs/feature-spec)
- [Railscasts #275: How I Test](https://www.youtube.com/watch?v=AQ-Vf157Ju8)
- [Rails 4 Test Perscriptions](https://pragprog.com/book/nrtest2/rails-4-test-prescriptions)
- [Capybara Cheatsheet](https://gist.github.com/zhengjia/428105)